### Clean the extracted FIG pdf data

In [107]:
import pandas as pd
import pyrfume
from pyrfume.odorants import get_cids
import re
# Load the data extracted by Tabula using the "Stream" method
df = pyrfume.load_data('IFRA_FIG/ifra-fragrance-ingredient-glossary---oct-2019.csv')

In [108]:
df = df.reset_index()
df.head()

,CAS number,Principal name,Primary descriptor,Descriptor 2,Descriptor 3
0,4221-98-1,(-)-(R)-.alpha.-Phellandrene,Aromatic,Medicinal,Citrus
1,512-13-0,(-)-.alpha.-Fenchol,Earthy,Woody,Camphoraceous
2,489-40-7,(-)-.alpha.-Gurjunene,Woody,Balsamic,Nutty
3,23986-74-5,(-)-Germacrene D,Woody,Spicy,Dry
4,489-86-1,(-)-Guaiol,Woody,Smoky,Rose


In [109]:
# These are the indices of overflow cells, which only contain the last few characters of the previous cell's molecule name
overflow_indices = df.index[df['CAS number'].isnull()]
overflow_indices

Int64Index([  21,   27,   36,  101,  158,  209,  564,  687,  757,  809,  828,
             890,  892,  894,  896,  967,  996, 1171, 1239, 1580, 1623, 1625,
            1746, 1751, 1753, 1755, 1900, 2524, 2526, 2544, 2710, 2820, 2923,
            2925, 2928, 2930, 2932],
           dtype='int64')

In [110]:
# Merge those last few characters into the previous cell's molecule name
for i in overflow_indices:
    df.loc[i-1, 'Principal name'] = '%s%s'% (df.loc[i-1, 'Principal name'], df.loc[i, 'Principal name'])

In [111]:
# Delete those overflow rows
df = df.loc[~df.index.isin(overflow_indices)]

In [112]:
# Fix problematic CAS numbers
for index, cas in df['CAS number'].items():
    if not re.match('[0-9]+\-[0-9]+\-[0-9]+', cas):
        print("Fixing %s" % cas)
        cas = cas.replace('(','').replace(')','')
        assert re.match('[0-9]+\-[0-9]+\-[0-9]+', cas)
        df.loc[index, 'CAS number'] = cas

Fixing (6876-12-6)


In [64]:
# Get CIDs for these CAS numbers
# Many of these CAS numbers are for substances, not compounds, and so have SIDs instead (not yet supported)
cas_cids_dict = get_cids(df['CAS number'])

[-----------------------100%---------------------] 3120 out of 3120 complete           
Could not find 102242-62-6
Could not find 1009814-14-5
Could not find 8006-81-3
Could not find 68952-44-3
Could not find 8006-81-3


Could not find 13837-56-4
Could not find 68877-29-2
Could not find 116044-44-1
Could not find 124071-43-8
Could not find 1309389-73-8
Could not find 1958027-44-5
Could not find 70266-48-7
Could not find 139504-68-0
Could not find 1333-52-4
Could not find 131812-52-7
Could not find 73018-51-6
Could not find 70131-51-0
Could not find 868846-58-6
Could not find 139539-67-6
Could not find 91069-37-3
Could not find 68555-94-2
Could not find 811436-82-5
Could not find 197098-61-6
Could not find 52711-52-1
Could not find 1801275-25-1
Could not find 1801275-26-2
Could not find 21662-22-6
Could not find 300371-33-9
Could not find 90480-35-6
Could not find 71048-83-4
Could not find 916887-53-1
Could not find 27043-05-6
Could not find 676125-00-1
Could not find 1127890-59-8
Could

{'4221-98-1': 7460,
 '512-13-0': 439711,
 '489-40-7': 16213731,
 '23986-74-5': 5317570,
 '489-86-1': 227829,
 '13837-56-4': 0,
 '1117-61-9': 101977,
 '4610-11-1': 6432154,
 '888021-82-7': 20656016,
 '17957-94-7': 442478,
 '2216-52-6': 439263,
 '67663-01-8': 106756,
 '156472-94-5': 9833996,
 '258823-39-1': 11240503,
 '58475-04-0': 21918450,
 '15892-23-6': 6568,
 '15932-80-6': 6988,
 '10138-32-6': 24986,
 '68877-29-2': 0,
 '35145-02-9': 56842481,
 '94400-98-3': 44152188,
 '389083-83-4': 22717736,
 '35836-73-8': 1272250,
 '35836-72-7': 929336,
 '68489-09-8': 11266244,
 '29461-13-0': 12217499,
 '5331-14-6': 79240,
 '23726-93-4': 5366074,
 '23726-91-2': 5374527,
 '67674-47-9': 6437117,
 '28069-72-9': 5362833,
 '116044-44-1': 0,
 '59739-63-8': 12309006,
 '81836-13-7': 44145973,
 '120811-92-9': 22336876,
 '124071-43-8': 0,
 '92046-48-5': 174845,
 '71172-26-4': 117035,
 '67845-46-9': 105764,
 '3288-99-1': 76784,
 '97358-54-8': 6366609,
 '39872-57-6': 6438067,
 '24720-09-0': 5366077,
 '22882-91

In [113]:
# Add CIDs to the dataframe
for cas, cid in cas_cids_dict.items():
    df.loc[df['CAS number']==cas, 'CID'] = cid
# Convert CIDs to integers
df.loc[:, 'CID'] = df.loc[:, 'CID'].astype(int)
df.head()

,CAS number,Principal name,Primary descriptor,Descriptor 2,Descriptor 3,CID
0,4221-98-1,(-)-(R)-.alpha.-Phellandrene,Aromatic,Medicinal,Citrus,7460
1,512-13-0,(-)-.alpha.-Fenchol,Earthy,Woody,Camphoraceous,439711
2,489-40-7,(-)-.alpha.-Gurjunene,Woody,Balsamic,Nutty,16213731
3,23986-74-5,(-)-Germacrene D,Woody,Spicy,Dry,5317570
4,489-86-1,(-)-Guaiol,Woody,Smoky,Rose,227829


In [114]:
# Use CID as the index and sort
df = df.set_index('CID').sort_index()
df.head()

,CAS number,Principal name,Primary descriptor,Descriptor 2,Descriptor 3
CID,,,,,
0,8006-77-7,Pimenta leaf oil,Spicy,Sweet,Balsamic
0,8008-52-4,"Coriander herb oil, rectified",Spicy,Fresh,Green
0,8007-01-0,Rose water oil,Floral,Rose,Green
0,8007-01-0,Rose tincture,Floral,Rose,Green
0,8007-01-0,Rose tincture,Floral,Rose,Green


In [116]:
pyrfume.save_data(df, 'IFRA_FIG/ifra_fig.csv')

In [118]:
pyrfume.load_data('IFRA_FIG/ifra_fig.csv')

,CAS number,Principal name,Primary descriptor,Descriptor 2,Descriptor 3
CID,,,,,
0,8006-77-7,Pimenta leaf oil,Spicy,Sweet,Balsamic
0,8008-52-4,"Coriander herb oil, rectified",Spicy,Fresh,Green
0,8007-01-0,Rose water oil,Floral,Rose,Green
0,8007-01-0,Rose tincture,Floral,Rose,Green
0,8007-01-0,Rose tincture,Floral,Rose,Green
...,...,...,...,...,...
92030006,476332-65-7,"2H-Indeno[4,5b] furan, decahydro-2,2,6,6,7,8,8...",Amber,Woody,Dry
92030009,1392277-05-2,"2H-2,4a-Methanonaphthalen-1(5H)-one, hexahydro...",Woody,Amber,Dry
102054965,900779-74-0,"1,5-Cyclododecadiene, 9-methoxy-1,6,10-trimethyl-",Woody,Amber,Leathery


In [125]:
from pyrfume.cabinets import get_mainland
df_mainland = get_mainland()
len(set(df_mainland['CAS']).intersection(df['CAS number']))

233

In [126]:
len(df_mainland.index.intersection(df.index))

257

In [129]:
df[df.index.isin(df_mainland.index)]#

,CAS number,Principal name,Primary descriptor,Descriptor 2,Descriptor 3
CID,,,,,
177,75-07-0,Acetaldehyde,Fruity,Apple,Aldehydic
179,513-86-0,Acetoin,Food Like,Buttery,Milky
240,100-52-7,Benzaldehyde,Fruity,Cherry,Almond
244,100-51-6,Benzyl alcohol,Balsamic,Fruity,Floral
261,123-72-8,Butyraldehyde,Gourmand,Fruity,Chocolate
...,...,...,...,...,...
5373603,17369-59-4,3-Propylidenephthalide,Herbal,Celery,Sweet
5374527,23726-91-2,"(2E)-1-(2,6,6-Trimethyl-1-cyclohexen-1-yl)-2-b...",Floral,Rose,Plum
6114390,120-24-1,Isoeugenyl phenylacetate,Herbal,Aromatic,Spicy


In [133]:
x = df_mainland.join(df, how='inner')[['CAS', 'CAS number', 'Primary descriptor', 'Descriptor 2', 'Descriptor 2']]

In [144]:
for cid in x.index:
    if cid>0:
        y = x.loc[cid, 'CAS'] != x.loc[cid, 'CAS number']
        if isinstance(y, bool):
            if y:
                print(cid)
        elif all(y):
            print(cid)

1254
1254
1549778


In [146]:
x.columns = ['Mainland CAS', 'FIG CAS', 'Primary descriptor', 'Descriptor 2', 'Descriptor 3']

In [148]:
x.to_csv('FIG-in-Mainland.csv')